In [ ]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time


def test_convergence(x):
    n,r = x

    nodes = []
    
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=1)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía
    
    return nodes
        
nodes = test_convergence((2,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 51873
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 58203
INFO:root:(('127.0.0.1', 51873)) Conexión aceptada con ('127.0.0.1', 58203)
INFO:root:(('127.0.0.1', 51873)) Broadcasting start learning...
INFO:root:(('127.0.0.1', 51873)) Sending Initial Model Weights
INFO:root:(('127.0.0.1', 51873)) Broadcasting model to 1 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 58203)) Broadcasting Number of Samples...
INFO:root:(('127.0.0.1', 58203)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 58203)) Model initialized
INFO:root:(('127.0.0.1', 51873)) Broadcasting Number of Samples...
INFO:root:(('127.0.0.1', 51873)) Waiting other node votes.
INFO:root:(('127.0.0.1', 58203)) Waiting other node votes.
('127.0.0.1', 51873) Train set: {('127.0.0.1', 58203): 128, ('127.0.0.1', 51873): 94}
('127.0.0.1', 58203) Train set: {('127.0.0.1', 58203): 128, ('127.0.0.1', 51873): 94}
INFO:root:(('12

GPU available: False, used: False


INFO:root:(('127.0.0.1', 58203)) Evaluating...


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
HPU available: False, using: 0 HPUs


Testing: 0it [00:00, ?it/s]

...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:root:(127.0.0.1:51873) Model added (1/2) from ('127.0.0.1', 51873)
INFO:root:(('127.0.0.1', 51873)) Broadcasting model to 1 clients. (size: 548864 bytes)
INFO:root:(127.0.0.1:58203) Model added (1/2) from ('127.0.0.1', 51873)
INFO:root:(127.0.0.1:51873) Agregating models. Timeout reached
INFO:root:(('127.0.0.1', 51873)) Waiting other nodes.
INFO:root:(127.0.0.1:58203) Agregating models. Timeout reached


In [ ]:
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
import threading

nodes = []
        
def test_node_down_on_learning(n):

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=2,epochs=0)

    # Stopping node
    nodes[1].stop()
    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break

    for node in nodes:
        node.stop()


nodes = []
for _ in range(6):
    test_node_down_on_learning(4)


Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 45675
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 57089
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 52805
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 46139
INFO:root:(('127.0.0.1', 45675)) Conexión aceptada con ('127.0.0.1', 57089)
INFO:root:(('127.0.0.1', 57089)) Conexión aceptada con ('127.0.0.1', 52805)
INFO:root:(('127.0.0.1', 52805)) Conexión aceptada con ('127.0.0.1', 45675)
INFO:root:(('127.0.0.1', 52805)) Conexión aceptada con ('127.0.0.1', 46139)
INFO:root:(('127.0.0.1', 46139)) Conexión aceptada con ('127.0.0.1', 57089)
INFO:root:(('127.0.0.1', 46139)) Conexión aceptada con ('127.0.0.1', 45675)
INFO:root:(('127.0.0.1', 45675)) Broadcasting start learning...
INFO:root:(('127.0.0.1', 45675)) Sending Initial Model Weights
INFO:root:(('127.0.0.1', 45675)) Broadcasting model to 3 clients. (size: 548864 bytes)
INFO:root:

GPU available: False, used: False


INFO:root:(('127.0.0.1', 46139)) Evaluating...


TPU available: False, using: 0 TPU cores
GPU available: False, used: False


INFO:root:(('127.0.0.1', 52805)) Evaluating...


...

Testing: 0it [00:00, ?it/s]

Exception in thread learning_thread-127.0.0.1:46139:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pedro/Documents/federated_learning_p2p/p2pfl/node.py", line 246, in __start_learning
    self.__train_step()
  File "/home/pedro/Documents/federated_learning_p2p/p2pfl/node.py", line 351, in __train_step
    self.__bc_metrics(self.__evaluate())
  File "/home/pedro/Documents/federated_learning_p2p/p2pfl/node.py", line 378, in __evaluate


Testing: 0it [00:00, ?it/s]

    retults = self.learner.evaluate()
  File "/home/pedro/Documents/federated_learning_p2p/p2pfl/learning/pytorch/lightninglearner.py", line 80, in evaluate
    results = self.trainer.test(self.model, self.data, verbose=False)
  File "/home/pedro/.local/share/virtualenvs/federated_learning_p2p-ODhT9q3L/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 938, in test
    return self._call_and_handle_interrupt(self._test_impl, model, dataloaders, ckpt_path, verbose, datamodule)
  File "/home/pedro/.local/share/virtualenvs/federated_learning_p2p-ODhT9q3L/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 723, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "/home/pedro/.local/share/virtualenvs/federated_learning_p2p-ODhT9q3L/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 985, in _test_impl
    results = self._run(model, ckpt_path=self.ckpt_path)
  File "/home/pedro/.local/share/virtualenvs/federa

INFO:root:(('127.0.0.1', 52805)) Evaluated. Losss: 2.301456928253174, Metric: 0.11810000240802765. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 52805)) Broadcasting metrics to 2 clients.
INFO:root:(('127.0.0.1', 52805)) Training...
INFO:root:(127.0.0.1:52805) Model added (1/3) from ('127.0.0.1', 52805)
INFO:root:(('127.0.0.1', 52805)) Broadcasting model to 2 clients. (size: 548864 bytes)
INFO:root:(127.0.0.1:45675) Model added (1/3) from ('127.0.0.1', 52805)
INFO:root:(127.0.0.1:46139) Model added (1/3) from ('127.0.0.1', 52805)
INFO:root:(('127.0.0.1', 45675)) Evaluated. Losss: 2.301456928253174, Metric: 0.11810000240802765. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 45675)) Broadcasting metrics to 2 clients.
INFO:root:(('127.0.0.1', 45675)) Training...
INFO:root:(127.0.0.1:45675) Model added (2/3) from ('127.0.0.1', 45675)
INFO:root:(('127.0.0.1', 45675)) Broadcasting model to 2 clients. (size: 548864 bytes)
INFO:root:(127.0.0.1:46139) Model added (2/3) 

In [2]:
#import threading
for thread in threading.enumerate(): 
    print(thread.name)

MainThread
IOPub
Heartbeat
Thread-3
Thread-4
Control
IPythonHistorySavingThread
Thread-2


In [6]:
import sys

def get_thread_position(thread):
    frame = sys._current_frames().get(thread.ident, None)
    if frame:
        return frame.f_code.co_filename, frame.f_code.co_name, frame.f_code.co_firstlineno

get_thread_position(threading.enumerate()[-1])

('/Users/pedro/Documents/UDC/4/TFG/federated_learning_p2p/p2pfl/node.py',
 '__on_round_finished',
 260)

In [1]:
TAIN_SET_NUM_NODESfrom p2pfl.const import HEARTBEAT_FREC, SOCKET_TIEMOUT
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
from test.fixtures import two_nodes, four_nodes


import threading
for thread in threading.enumerate(): 
    print(thread.name)
    
n1 = Node(MLP(),MnistFederatedDM())
n2 = Node(MLP(),MnistFederatedDM())
print("\n")
def test_bad_binary_model():

    n1.start()
    n2.start()

    n1.connect_to(n2.host,n2.port)
    time.sleep(0.1) 

    # Start Learning
    n1.set_start_learning(rounds=2,epochs=0)
 
    # Adding noise to the buffer
    for _ in range(200):self.neightboors
        n1.neightboors[0].param_bufffer += "noise".encode("utf-8")
    
    while not n1.round is None and not n2.round is None:
        time.sleep(0.1)
        
    n1.stop()
    n2.stop()
    
test_bad_binary_model()

time.sleep(10)

for thread in threading.enumerate(): 
    print(thread.name)
    
print("\n")



ModuleNotFoundError: No module named 'p2pfl.const'

In [4]:

for thread in threading.enumerate(): 
    print(thread.name)
    

MainThread
IOPub
Heartbeat
Thread-3
Thread-4
Control
IPythonHistorySavingThread
Thread-2


In [6]:
print(n1.round)

None


In [4]:
from p2pfl.settings import Settings

Settings.AGREGATION_TIEMOUT = 1

In [5]:
"0.1545318067073822".isdecimal()

False

In [4]:
message = ['METRICS', '0', '2.301456928253174', '0.13700607419013977']
print(int(message[1]), float(message[2]), float(message[3]))

0 2.301456928253174 0.13700607419013977


In [11]:
import random
import math
a = [1,2,3,4,5,6,7,8,9,10]

random.choices(a, k=3)

[5, 8, 2]

In [13]:
[random.randint(0,1000),math.floor(random.randint(0,1000)/2),math.floor(random.randint(0,1000)/4)]

[300, 298, 15]

In [15]:
"".join(("1","2"))

'12'

In [20]:
def build_vote_train_set_msg(candidates, weights):
    aux = ""
    for i in range(len(candidates)):
        aux = aux + " " + " ".join(candidates[i]) + " " + str(weights[i])
    return ("CommunicationProtocol.VOTE_TRAIN_SET" + aux + "\n").encode("utf-8")

candidates = [("host","port"),("host","port"),("host","port")]
weights = [random.randint(0,1000),math.floor(random.randint(0,1000)/2),math.floor(random.randint(0,1000)/4)]

build_vote_train_set_msg(candidates,weights)

b'CommunicationProtocol.VOTE_TRAIN_SET host port 443 host port 194 host port 245\n'

In [2]:
[str(x) for x in ("1",2)]

['1', '2']

In [4]:
a=[(1,2),(1,2)]
b=[1,2]
print(list(zip(a,b)))

[((1, 2), 1), ((1, 2), 2)]


In [2]:
list("[]")

['[', ']']

In [4]:
[1,2,3,4].index(9)

ValueError: 9 is not in list

In [5]:
[1,2,3,4,5,6][2:]

[3, 4, 5, 6]

In [7]:
5%2

1

In [9]:
all([True,[False]])

True

In [1]:
{
    (1,2):3
}

{(1, 2): 3}

In [15]:
dicta = {('127.0.0.1', 47605): 165}

for i in range(len(dicta)):
    print(list(dicta.keys())[i])


('127.0.0.1', 47605)


In [14]:
list(dicta.keys())

[('127.0.0.1', 47605)]

In [4]:
for n in nodes:
    for nc in n.neightboors:
        
        print(nc.get_train_set_votes())

{('127.0.0.1', 34535): 186}
[]


In [2]:
len({('127.0.0.1', 51977): 149})

1